In [45]:
from PIL import Image
import hashlib
import random

In [46]:
# Открыть BMP-файл и загрузить его содержимое
image = Image.open("28.bmp")
pixels = image.load()

In [47]:
# Сгенерировать ключ – список из случайных позиций байтов, в которые будут прятаться фейковые биты
key = random.sample(range(122, image.size[0] * image.size[1] * 3), 8 * len(hashlib.sha1('leasing.txt'.encode()).digest()))

In [48]:
# Внедрить хешкод файла в контейнеры BMP-файла
binary_hash = ''.join(format(x, '08b') for x in hashlib.sha1('leasing.txt'.encode()).digest())
i = 0
for byte in binary_hash:
    byte_position = key[i]
    x = (byte_position - 122) // 3 % image.size[0]
    y = (byte_position - 122) // 3 // image.size[0]
    pixel = list(pixels[x, y])
    pixel[byte_position % 3] = (pixel[byte_position % 3] & 0xFE) | int(byte)
    pixels[x, y] = tuple(pixel)
    i += 1

In [49]:
# Сохранить модифицированный BMP-файл
image.save("28_modified.bmp")

# Извлечь хеш-код файла из контейнеров BMP-файла
extracted_hash = ''
for i in range(8 * len(hashlib.sha1('leasing.txt'.encode()).digest())):
    byte_position = key[i]
    x = (byte_position - 122) // 3 % image.size[0]
    y = (byte_position - 122) // 3 // image.size[0]
    pixel = pixels[x, y]
    extracted_hash += str(pixel[byte_position % 3] & 0x01)
extracted_hash = hex(int(extracted_hash, 2))[2:]

In [50]:
# Сравнить извлеченный хеш-код с оригинальным хешкодом файла и проверить, что они совпадают
original_hash = hashlib.sha1('leasing.txt'.encode()).hexdigest()
if extracted_hash == original_hash:
    print("Хеш-коды совпадают")
else:
    print("Хеш-коды не совпадают")


Хешкоды совпадают
